In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import numpy as np

In [2]:
data = pd.read_csv("./data/망고플레이트전국댓글.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658251 entries, 0 to 658250
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   리뷰내용    657292 non-null  object
 1   작성자명    657680 non-null  object
 2   후기점수    658251 non-null  object
 3   url     658251 non-null  object
 4   ID      658251 non-null  object
dtypes: object(5)
memory usage: 25.1+ MB


In [4]:
data.head()

,리뷰내용,작성자명,후기점수,url,ID
0,맛있네요,-,맛있다,https://www.mangoplate.com/restaurants/iMRRP69...,iMRRP69qtkeO
1,가로수길,전예원,맛있다,https://www.mangoplate.com/restaurants/iMRRP69...,iMRRP69qtkeO
2,맛있어요,전예원,맛있다,https://www.mangoplate.com/restaurants/iMRRP69...,iMRRP69qtkeO
3,언제와도 맛있는 미라이!!!고등어산도는 무조건 시켜야하구요.. (식빵 뭔지 궁금) ...,스틱스강에맹세하는맛집,맛있다,https://www.mangoplate.com/restaurants/iMRRP69...,iMRRP69qtkeO
4,"아와이 야끼소바,사바산도,우니 아마에비 호다테먹었어요.나마비루도 함께^^나마비루 부...",ㅇㅇ,맛있다,https://www.mangoplate.com/restaurants/iMRRP69...,iMRRP69qtkeO


In [5]:
data['ID'].nunique()

5744

In [6]:
data = data.drop_duplicates(['리뷰내용','작성자명'])

In [7]:
review_counts = data.groupby('ID')[['리뷰내용']].count()

In [8]:
review_counts.sort_values(by='리뷰내용', ascending=False)

,리뷰내용
ID,
s6C_cdBQX0rj,372
1pmv75vVQx,287
fPi6OZ3r-u,246
y-_XMRSTNrAi,241
gL8RksQTNk,239
...,...
nwykgqfbYx,1
AzIdNOqU_tl3,1
noi8A_NGrRBL,1


In [9]:
over20reviews = review_counts[review_counts['리뷰내용'] > 20]

In [10]:
over20reviews

,리뷰내용
ID,
-1rMxmPWvg,32
-1yzcgbtF5,96
-9jLnAAJZc,25
-D6qxV9cg9mT,23
-EDDtVfafvaS,29
...,...
zYzhPaX003,69
z_sD7nAsh7V0,51
zq9bIKh6PMLV,37


In [11]:
over20reviews_shops = over20reviews.index

In [12]:
over20reviews_shops

Index(['-1rMxmPWvg', '-1yzcgbtF5', '-9jLnAAJZc', '-D6qxV9cg9mT',
       '-EDDtVfafvaS', '-G-YWwpe9D', '-LWflJV1maU2', '-SMNjUsH3i',
       '-SvQDki5HNzQ', '-UUbd93wKH0H',
       ...
       'zNVXE1__XF', 'zOOlz9vfJfFF', 'zSujg5KWKY1W', 'zSxicuDvNH',
       'zWVgSFqWIQrj', 'zYzhPaX003', 'z_sD7nAsh7V0', 'zq9bIKh6PMLV',
       'ztTTr81jaBm7', 'zv1rcsrmRU4S'],
      dtype='object', name='ID', length=1117)

In [13]:
"s6C_cdBQX0rj" in over20reviews_shops

True

In [14]:
over20df = data[data['ID'].apply(lambda x: x in over20reviews_shops)]

In [15]:
over20df['리뷰내용'] = over20df['리뷰내용'].astype('str')

C:\Users\HSM\AppData\Local\Temp\ipykernel_5900\3256979473.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  over20df['리뷰내용'] = over20df['리뷰내용'].astype('str')


In [16]:
over20df = over20df[over20df['리뷰내용'].apply(len) >= 4]

In [17]:
over20df['ID'].nunique()

1117

In [18]:
over20df['ID'] = over20df['ID'].str.replace("-","_")

In [19]:
def dfbyid(over20df):
    dfs = []
    for index, id in enumerate(over20df['ID']):
        globals()[f"d_{id}"] = over20df[over20df['ID'] == id]   # 변수를 자동으로 생성해서 id별 dataframe만듬
        dfs.append("d_"+id) # ID가 숫자로 시작하면 변수 생성이 안되므로 앞에 d_를 붙임
    dfs = set(dfs) # ID 중복 제거
    print(f"{index+1}/{len(over20df['ID']} 처리중", end="\r") # 현재 진행사항 표시
    return list(dfs)
    

In [20]:
dfs = dfbyid(over20df)

In [21]:
len(dfs)

1117

In [22]:
# pip install emoji

In [23]:
def wcByShop(dfs):
    import string
    from emoji import core
    from wordcloud import WordCloud

    font_path = r"C:\Windows\Fonts\gulim.ttc"
    for idx, val in enumerate(dfs):
        globals()[f'{val}_text'] = []
        globals()[val]['리뷰내용'].apply(lambda x: globals()[f'{val}_text'].append(x))
        globals()[f'{val}_text'] = " ".join(globals()[f'{val}_text'])
        globals()[f'{val}_text'] = globals()[f'{val}_text'].translate(str.maketrans(".", " ", string.punctuation)).replace("…", " ")
        globals()[f'{val}_text'] = core.replace_emoji(globals()[f'{val}_text'], replace=" ").replace("  ", " ")
        globals()[f'{val}_text'] = globals()[f'{val}_text'].translate(str.maketrans("", "", "ㅋㅎㅇ"))
        
        # wordcloud 만들기
        wc = WordCloud(
        max_words = 200,
        font_path= font_path,
        background_color = 'white',
        stopwords = []).generate(globals()[f'{val}_text'])
        plt.figure(figsize=(10,10))
        plt.imshow(wc)
        plt.axis('off')
        plt.savefig(f"./data/wc/{val}.png")
        plt.close()
        print(f"현재 {idx+1}/{len(dfs)} 처리중", end="\r" )


In [24]:
# id 별로 저장된 데이터 프레임을 wcByShop함수를 이용해서 텍스트 하나의 변수에 합치기, 텍스트 클리닝, 워드클라우드 만들어 저장하기 수행
wcByShop(dfs)